# 基于BERT + 余弦相似度的文本相似度计算

## 导入所需库，创建设备对象

In [1]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

## 简单示例

In [2]:
# 1. 加载中文 BERT 模型和分词器
tokenizer = BertTokenizer.from_pretrained('../bert-base-chinese/')
model = BertModel.from_pretrained('../bert-base-chinese')

# 2. 将模型移动到GPU上（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 3. 准备两个中文句子
text1 = "我喜欢机器学习和自然语言处理"
text2 = "我热爱人工智能和深度学习"

# 4. 对句子进行分词和编码，并将输入数据移动到相同的设备上
inputs1 = tokenizer(text1, return_tensors='pt', padding=True, truncation=True, max_length=128).to(device)
inputs2 = tokenizer(text2, return_tensors='pt', padding=True, truncation=True, max_length=128).to(device)

# 5. 获取 BERT 输出（取 [CLS] 向量作为句子表示）
with torch.no_grad():
    outputs1 = model(**inputs1)
    outputs2 = model(**inputs2)

# 取 [CLS] token 的向量（句子的全局表示），并转换为numpy数组
sentence_vector1 = outputs1.last_hidden_state[:, 0, :].squeeze().cpu().numpy()
sentence_vector2 = outputs2.last_hidden_state[:, 0, :].squeeze().cpu().numpy()

# 6. 计算余弦相似度
similarity = cosine_similarity([sentence_vector1], [sentence_vector2])

# 7. 输出结果
print(f"文本1: {text1}")
print(f"文本2: {text2}")
print(f"文本相似度（基于 BERT）: {similarity[0][0]:.4f}")

文本1: 我喜欢机器学习和自然语言处理
文本2: 我热爱人工智能和深度学习
文本相似度（基于 BERT）: 0.8821


## 测试

In [6]:
from src.bert_utils import bert_similarity_metric


from dspy import Example

# 示例 gold 数据（标注样本）
gold = Example({
    'input': '西北实习生李龙。你好天山站，杨立斌向您回令。喂啥那个啥？是那个操操作天哈，一线线路保护投入的。保护投入了稍等我给你转一下。嗯，好。',
    'fault_equipment': '无',
    'fault_time': '无',
    'region': '无',
    'voltage_level': '无',
    'weather_condition': '无',
    'fault_reason_and_check_result': '无',
    'fault_recovery_time': '无',
    'illustrate': '天哈，一线线路保护投入',
    'line_name': '天哈，一线线路',
    'power_supply_time': '无',
    'fault_phase': '无',
    'protect_info': '无',
    'plant_station_name': '天山站',
    'bus_name': '无',
    'bus_type': '无',
    'handling_status': '无',
    'detailed_description': '无',
    'expecteddefect_elimination_time': '无',
    'protection_action': '操操作天哈，一线线路保护投入',
    'trip_details': '无',
    'unit_num': '无',
    'manufacturer': '无',
    'production_date': '无'
}).with_inputs('input')  # 指定 input 为输入字段

# 示例 pred 数据（模型预测结果）
pred = Example({
    'input': '西北实习生李龙。你好天山站，杨立斌向您回令。喂啥那个啥？是那个操操作天哈，一线线路保护投入的。保护投入了稍等我给你转一下。嗯，好。',
    'fault_equipment': '无',
    'fault_time': '无',
    'region': '无',
    'voltage_level': '无',
    'weather_condition': '无',
    'fault_reason_and_check_result': '无',
    'fault_recovery_time': '无',
    'illustrate': '天哈一线线路保护投入',
    'line_name': '天哈一线线路',
    'power_supply_time': '无',
    'fault_phase': '无',
    'protect_info': '无',
    'plant_station_name': '天山电站',
    'bus_name': '无',
    'bus_type': '无',
    'handling_status': '无',
    'detailed_description': '无',
    'expecteddefect_elimination_time': '无',
    'protection_action': '操作天哈一线线路保护投入',
    'trip_details': '无',
    'unit_num': '无',
    'manufacturer': '无',
    'production_date': '无'
}).with_inputs('input')  # 指定 input 为输入字段

# 计算语义相似度
score = bert_similarity_metric(gold, pred)
print(f"平均语义相似度：{score:.4f}")

平均语义相似度：0.9887
